In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("dataset/finalfeatures.csv")

In [10]:
df.head()

,Unnamed: 0,skip_2,session_position,context_switch,no_pause_before_play,hist_user_behavior_n_seekfwd,hist_user_behavior_n_seekback,hist_user_behavior_is_shuffle,hour_of_day,context_type,...,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration,time_signature
0,0,0,1,0,0,0,0,1,16,1,...,1,-6.577,1,0.069717,3.484468e-03,0.678553,0.152255,134.024994,180066.665649,4
1,1,0,2,0,1,0,0,1,16,1,...,7,-5.319,0,0.061158,1.031315e-07,0.104322,0.337152,130.037994,236796.371460,4
2,2,0,3,0,1,0,0,1,16,1,...,10,-5.843,1,0.045354,2.659035e-08,0.135776,0.373862,145.028000,231266.662598,4
3,3,0,4,0,1,0,0,1,16,1,...,1,-7.756,1,0.229936,6.598388e-06,0.103722,0.649420,111.982002,169826.675415,4
4,4,0,5,0,1,0,0,1,16,1,...,8,-4.919,1,0.240980,2.066649e-06,0.120842,0.652921,147.031006,210545.257568,4


In [3]:
from datetime import datetime 
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import seaborn as sns

In [4]:
x = df.drop(['skip_2'],axis = "columns")
y = df.skip_2

In [5]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = .20 , random_state = 42)

In [6]:
x_train.shape,y_train.shape,x_test.shape,y_test.shape

((134304, 23), (134304,), (33576, 23), (33576,))

In [7]:
import lightgbm as lgb

d_train = lgb.Dataset(x_train, label=y_train)

In [8]:
lgbm_params = {'learning_rate':0.23, 'boosting_type':'dart',
              'objective':'binary',
              'metric':['auc', 'binary_logloss'],
              'num_leaves':2500,
              'max_depth':5,
              "min_data_in_leaf": 200,
              "lambda_l1": 15,
              "lambda_l2": 70,
              "min_gain_to_split": 2,
              "bagging_fraction": 0.7,
              "bagging_freq": 1,
              "feature_fraction": 0.9}

In [9]:
from datetime import datetime
start=datetime.now()
clf = lgb.train(lgbm_params, d_train, 200) #50 iterations. Increase iterations for small learning rates
stop=datetime.now()
execution_time_lgbm = stop-start
print("LGBM execution time is: ", execution_time_lgbm)

LightGBMError: Do not support special JSON characters in feature name.

In [ ]:
y_pred_lgbm=clf.predict(x_test)

#convert into binary values 0/1 for classification
for i in range(0, x_test.shape[0]):
    if y_pred_lgbm[i]>=.5:       # setting threshold to .5
       y_pred_lgbm[i]=1
    else:  
       y_pred_lgbm[i]=0
       
#Print accuracy
print ("Accuracy with LGBM = ", metrics.accuracy_score(y_pred_lgbm,y_test))

In [ ]:
# import pickle

# filename = 'models/gbt3.pkl'
# pickle.dump(clf, open(filename, 'wb'))